In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/guide_reference-Kat7_dual_guide.csv')
df = df.rename(columns={'gRNA_complete': 'gRNA'})

In [3]:
# rename the original dataframe
df['Gene'] = df['Gene'].str.replace('Curate_', '', regex=False)
df['Gene'] = df['Gene'].str.replace('safe', 'Safe', regex=False)
df['Guide'] = df['Gene']
mask = df['Gene'].duplicated(keep=False)
df.loc[mask, 'Guide'] += '_'+df.groupby(
    'Guide').cumcount().add(1).astype(str)
df.loc[df['Gene'].str.contains('NT|Safe'),'Guide'] = df.loc[df['Gene'].str.contains('NT|Safe'),'Gene']

In [4]:
# seperate experimental guide and inert guide
df_sub1 = df[~df.Gene.str.contains('Spikein')].copy()
df_sub1.head()

,Gene,gRNA,Position,Guide
0,NT_1,CACGGGCAGCGATCTAGTTG,G1,NT_1
1,NT_2,CTTCACGCCTTGGACCGATA,G1,NT_2
2,Safe_16,AGTACACTATAGTGGCTAGT,G1,Safe_16
3,Safe_37,ATGTAGGAGCTATAACATC,G1,Safe_37
4,Ing5,CTAGACCAGAGAACAGAAGG,G1,Ing5_1


In [5]:
df_sub2 = df[df.Gene.str.contains('Spikein')].copy()
df_sub2 

,Gene,gRNA,Position,Guide
40,Spikein_1,ACCTTACAATGTTATATT,G1,Spikein_1
41,Spikein_2,GTGACACTTTCTATTAGGT,G1,Spikein_2
42,Spikein_3,CCTTGAAATCAAATCAAACC,G1,Spikein_3
43,Spikein_4,GACTAGCAGGTGACACCACCTGCATCA,G2,Spikein_4


In [6]:
temp_sgRNA = df_sub1
df_P1 = temp_sgRNA[temp_sgRNA['Position'] == 'G1'].copy()
df_P2 = temp_sgRNA[temp_sgRNA['Position'] == 'G2'].copy()
df_P1['key'] = 1
df_P2['key'] = 1

# Perform the merge to get all combinations
df_combined = pd.merge(df_P1.drop(columns='Position'), df_P2.drop(columns='Position'), on='key', suffixes=('1','2')).drop('key', axis=1)
df_combined['gRNA_combination'] = df_combined.gRNA1+'_'+df_combined.gRNA2
df_combined_sub1 = df_combined.copy()
df_combined_sub1['Array_category'] = 'Kat7'

In [7]:
temp_sgRNA = df_sub2
df_P1 = temp_sgRNA[temp_sgRNA['Position'] == 'G1'].copy()
df_P2 = temp_sgRNA[temp_sgRNA['Position'] == 'G2'].copy()

df_P1['key'] = 1
df_P2['key'] = 1

# Perform the merge to get all combinations
df_combined = pd.merge(df_P1.drop(columns='Position'), df_P2.drop(columns='Position'), on='key', suffixes=('1','2')).drop('key', axis=1)
df_combined['gRNA_combination'] = df_combined.gRNA1+'_'+df_combined.gRNA2
df_combined_sub2 = df_combined.copy()
df_combined_sub2['Array_category'] = 'Spikein'

In [8]:
df_combined = pd.concat([df_combined_sub1,df_combined_sub2],ignore_index=True)

In [9]:
df_combined

,Gene1,gRNA1,Guide1,Gene2,gRNA2,Guide2,gRNA_combination,Array_category
0,NT_1,CACGGGCAGCGATCTAGTTG,NT_1,NT_1,CACGGGCAGCGATCTAGTTG,NT_1,CACGGGCAGCGATCTAGTTG_CACGGGCAGCGATCTAGTTG,Kat7
1,NT_1,CACGGGCAGCGATCTAGTTG,NT_1,NT_2,CTTCACGCCTTGGACCGATA,NT_2,CACGGGCAGCGATCTAGTTG_CTTCACGCCTTGGACCGATA,Kat7
2,NT_1,CACGGGCAGCGATCTAGTTG,NT_1,Safe_16,AGTACACTATAGTGGCTAGT,Safe_16,CACGGGCAGCGATCTAGTTG_AGTACACTATAGTGGCTAGT,Kat7
3,NT_1,CACGGGCAGCGATCTAGTTG,NT_1,Safe_37,ATGTAGGAGCTATAACATC,Safe_37,CACGGGCAGCGATCTAGTTG_ATGTAGGAGCTATAACATC,Kat7
4,NT_1,CACGGGCAGCGATCTAGTTG,NT_1,Epc1,CTATGAGTCCATATACCCTG,Epc1_1,CACGGGCAGCGATCTAGTTG_CTATGAGTCCATATACCCTG,Kat7
...,...,...,...,...,...,...,...,...
382,Meaf6,CCAATAGCAAAAACGACCGG,Meaf6_2,Psip1,AAAAATTTAGCTAAACCAG,Psip1_1,CCAATAGCAAAAACGACCGG_AAAAATTTAGCTAAACCAG,Kat7
383,Meaf6,CCAATAGCAAAAACGACCGG,Meaf6_2,Psip1,ACTCCAAAAGCTGCCAGGCG,Psip1_2,CCAATAGCAAAAACGACCGG_ACTCCAAAAGCTGCCAGGCG,Kat7
384,Spikein_1,ACCTTACAATGTTATATT,Spikein_1,Spikein_4,GACTAGCAGGTGACACCACCTGCATCA,Spikein_4,ACCTTACAATGTTATATT_GACTAGCAGGTGACACCACCTGCATCA,Spikein
385,Spikein_2,GTGACACTTTCTATTAGGT,Spikein_2,Spikein_4,GACTAGCAGGTGACACCACCTGCATCA,Spikein_4,GTGACACTTTCTATTAGGT_GACTAGCAGGTGACACCACCTGCATCA,Spikein


In [10]:
df_combined.to_csv('data/guide_reference-kat7_array.csv',index=False)